In [1]:
import os, tqdm, time, random
import cv2 as cv
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import models, layers, losses, optimizers

from loss import *
from utils import *
from network import *

random.seed(777)
tf.set_random_seed(777)
np.random.seed(777)

In [2]:
root_dir = '../data/for_section/'

fold_idx = 1

train_path = os.path.join(root_dir, 'for_fold_%d'%fold_idx)

val_path = os.path.join(root_dir, 'for_fold_%d_val'%fold_idx)

In [3]:
test = data_loader(train_path)

In [4]:
data = []
label = []
n_size = 32
n_slice = 16
cs_strides = 16
a_strides = 2
cnt = 0
cor, sag, axi = test.shape
#print(test.shape)
for idx in tqdm.tqdm_notebook(range(0, cor-n_size, cs_strides)):
    for jdx in range(0, sag-n_size, cs_strides):
        for kdx in range(0, axi-(n_slice*6), a_strides):
            tmp = test[idx:idx+n_size, jdx:jdx+n_size, kdx:kdx+(n_slice*6)]
            if tmp.mean() > 225:
                label.append(tmp)
                tmp = np.array(np.dsplit(tmp, n_slice))
                tmp = tmp.mean(axis=-1)
                tmp = np.transpose(tmp, [1,2,0])
                data.append(tmp)
                
label = np.array(label)[..., np.newaxis]
data = np.array(data)[..., np.newaxis]
print(data.shape)
print(label.shape)


(3317, 32, 32, 16, 1)
(3317, 32, 32, 96, 1)


In [24]:
date = time.ctime().split(' ')

mon = 'Sep'
day = 17
year = 2019
residual_channels = 64
ckpt_root = './checkpoint/%s_%02d_%d/RBSR%d_msegrad_test'%(mon, day, year, residual_channels)
result_root = './result/%s_%02d_%d/RBSR%d_msegrad_test'%(mon, day, year, residual_channels)

with open(os.path.join(ckpt_root, "model.json"), "r") as json_file:
    model_json = json_file.read()
print("\nModel Read!\n")


Model Read!



In [8]:
net = models.model_from_json(model_json)
net.load_weights(os.path.join(ckpt_root, 'weight.h5'))

W0919 04:56:29.202686 139930111289088 deprecation.py:506] From /root/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0919 04:56:29.203863 139930111289088 deprecation.py:506] From /root/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0919 04:56:29.205344 139930111289088 deprecation.py:506] From /root/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from

In [10]:
scan_list = sorted(os.listdir(val_path))[1:]
for scan in scan_list:
    dante_path = os.path.join(val_path, scan, 'T1SPACE09mmISOPOSTwDANTE')
    img_name = [i for i in os.listdir(dante_path) if '.nii' in i and '_rsl' not in i][0]
    #print(img_name)
    val = nib.load(os.path.join(dante_path, img_name))
    val = check_data(val.get_data())
    
val_in = []
cor, sag, axi = val.shape
tmp = np.array(np.dsplit(val, axi//6))
tmp = tmp.mean(axis=-1)
tmp = np.transpose(tmp, [1, 2, 0])

#half_top = int(np.floor(tmp.shape[-1]/2))
#half_bot = int(np.ceil(tmp.shape[-1]/2))

In [55]:
recon = np.zeros(shape=[cor, sag, axi])

slice_dict = {
    1:[[0, 160], [0, 160], [0, 128], [0, 128]], 
    2:[[0, 160], [96, 256], [0, 128], [32, 256]], 
    3:[[96, 256], [0, 160], [32, 256], [0, 128]],
    4:[[96, 256], [96, 256], [32, 256], [32, 256]]
}
test = {}
sli = 12
i = 0
for row in range(2):
    row_start = row*128
    for col in range(2):
        
        col_start = col*128
        print(i, row_start, col_start)
        test[i] = net.predict(tmp[np.newaxis, 
                                   slice_dict[i+1][0][0]:slice_dict[i+1][0][1],
                                   slice_dict[i+1][1][0]:slice_dict[i+1][1][1],
                                   :, np.newaxis])
        recon[row_start:row_start+128, 
              col_start:col_start+128] = test[i][0,
                                                  slice_dict[i+1][2][0]:slice_dict[i+1][2][1],
                                                  slice_dict[i+1][3][0]:slice_dict[i+1][3][1], :, 0]
        i += 1
# test_3d_2 = net.predict(tmp[np.newaxis, ..., half_top:, np.newaxis])

0 0 0
1 0 128
2 128 0
3 128 128


In [56]:
aff = np.eye(4)
aff[2, 2]=6

nib.save(nib.Nifti1Image(tmp, aff), os.path.join(result_root, 'val_input.nii'))
nib.save(nib.Nifti1Image(recon, np.eye(4)), os.path.join(result_root, 'val_pred.nii'))
nib.save(nib.Nifti1Image(val, np.eye(4)), os.path.join(result_root, 'val_label.nii'))